# Что делает этот ноутбук

Этот ноутбук - часть разработки AI-продукта, который должен улучшать пользовательские промты, генерировать несколько вариантов изображений и автоматически выбирать лучший результат (или несколько лучших - в зависимости от режима работы).

В продукте будет две крупные части:

1. **Модуль улучшения промта**  
   Принимает сырой текст от пользователя, переписывает/расширяет его, подбирает стиль, добавляет технические детали для генерации.

2. **Модуль судьи (этот ноутбук)**  
   Получает уже сгенерированные картинки и их промты и выставляет им оценки, чтобы выбрать лучшее изображение для вывода пользователю.

В данном ноутбуке представлены baseline решения для модуля улучшения промта будущего продукта.

## Улучшение промтов

Модель должна

- принимаеть сырые русские промты,
- очищать, нормализовать и структурировать их,
- приводить к единому корпоративному стилю (brand-style),
- переписывать в виде длинного, аккуратного, информативного английского промта.

In [37]:
import re
import emoji
import json

import pandas as pd

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

from tqdm import tqdm

In [2]:
df = pd.read_csv(r"../datasets/prompt_dataset.csv")
df.head()

,prompts,prepared_reference
0,красивая картинка для сторис,"Create a clean, visually appealing Instagram S..."
1,сделать фон поярче,"Design a bright, vibrant background with smoot..."
2,фотка для рекламы одежды,Produce a stylish fashion advertisement image ...
3,стильный баннер срочно,"Generate a sleek, modern promotional banner wi..."
4,летняя распродажа визуал,"Create a summer sale visual with warm colors, ..."


In [3]:
BRAND_STYLE_SHORT = (
    "modern minimalistic premium visual, soft purple gradients, "
    "core color #7D3EFF, deep violet shadows, clean composition, "
    "digital glossy accents, high-end commercial aesthetic"
)

In [4]:
def clean_text(prompt: str) -> str:
    p = str(prompt).strip()
    p = emoji.replace_emoji(p, replace="")
    p = re.sub(r"<.*?>", "", p)
    p = re.sub(r"[^a-zA-Zа-яА-Я0-9ёЁ ,.!?%+\-]", " ", p)
    p = re.sub(r"([.!?,])\1+", r"\1", p)
    p = re.sub(r"\s+", " ", p).strip()
    p = p.lower()

    return p

In [5]:
def normalize_case(prompt: str) -> str:
    if not prompt:
        return prompt
    p = prompt.strip()
    return p[0].upper() + p[1:].lower()

In [6]:
def detect_platform(prompt: str) -> str:
    p = prompt.lower()

    if "сторис" in p or "stories" in p or "story" in p:
        return "instagram_story"
    if "reels" in p or "рилс" in p:
        return "instagram_reels"
    if "инста" in p or "instagram" in p:
        return "instagram_post"
    if "тикток" in p or "tiktok" in p or "тт" in p:
        return "tiktok_video"
    if "ютуб" in p or "youtube" in p:
        if "обложк" in p or "thumbnail" in p:
            return "youtube_thumbnail"
        return "youtube_video"
    if "pinterest" in p or "пинтерест" in p:
        return "pinterest_pin"
    if "вк" in p or "vk" in p or "вконтакт" in p:
        return "vk_post"
    if "тг" in p or "телеграм" in p or "telegram" in p:
        return "telegram_post"
    if "баннер" in p or "банер" in p or "banner" in p:
        return "banner"
    if "сайт" in p or "лендинг" in p or "landing" in p:
        return "website_visual"
    if "обложка" in p:
        return "cover_art"
    
    return "generic"


def detect_goal(prompt: str) -> str:
    p = prompt.lower()

    if any(word in p for word in ["реклама", "промо", "акция", "sale", "скидк"]):
        return "promotion"
    if any(word in p for word in ["бренд", "фирмен", "стиль", "айдентика"]):
        return "branding"
    if any(word in p for word in ["товар", "продукт", "предмет", "каталог", "ecom", "e-commerce"]):
        return "product_shot"
    if any(word in p for word in ["фон", "background", "подложка"]):
        return "background"
    if any(word in p for word in ["пост", "картинка", "визуал", "креатив"]):
        return "general_visual"
    if any(word in p for word in ["beauty", "косметика", "макияж", "скинкеар", "skincare"]):
        return "beauty"
    if any(word in p for word in ["фэшн", "fashion", "одежда", "лук", "стиль"]):
        return "fashion"
    if any(word in p for word in ["технолог", "tech", "стартап", "startup"]):
        return "tech_visual"
    if any(word in p for word in ["новогод", "праздник", "holiday", "зимн", "летн"]):
        return "seasonal"
    if any(word in p for word in ["еда", "food", "ресторан", "доставка"]):
        return "food_and_beverage"
    
    return "general_visual"

In [7]:
def preprocess_prompt_row(row: pd.Series) -> pd.Series:
    raw = row["prompts"]
    cleaned = clean_text(raw)

    platform = detect_platform(cleaned)
    goal = detect_goal(cleaned)

    return pd.Series({
        "raw_prompt": raw,
        "cleaned_prompt": cleaned,
        "platform": platform,
        "goal": goal,
        "brand_style": BRAND_STYLE_SHORT,
        "prepared_reference": row.get("prepared_reference", None),
        "cleaned_human": normalize_case(cleaned),
    })

In [8]:
df_processed = df.apply(preprocess_prompt_row, axis=1)
df_processed.head()

,raw_prompt,cleaned_prompt,platform,goal,brand_style,prepared_reference,cleaned_human
0,красивая картинка для сторис,красивая картинка для сторис,instagram_story,general_visual,"modern minimalistic premium visual, soft purpl...","Create a clean, visually appealing Instagram S...",Красивая картинка для сторис
1,сделать фон поярче,сделать фон поярче,generic,background,"modern minimalistic premium visual, soft purpl...","Design a bright, vibrant background with smoot...",Сделать фон поярче
2,фотка для рекламы одежды,фотка для рекламы одежды,generic,general_visual,"modern minimalistic premium visual, soft purpl...",Produce a stylish fashion advertisement image ...,Фотка для рекламы одежды
3,стильный баннер срочно,стильный баннер срочно,banner,branding,"modern minimalistic premium visual, soft purpl...","Generate a sleek, modern promotional banner wi...",Стильный баннер срочно
4,летняя распродажа визуал,летняя распродажа визуал,generic,general_visual,"modern minimalistic premium visual, soft purpl...","Create a summer sale visual with warm colors, ...",Летняя распродажа визуал


In [9]:
def interactive_prompt_preprocessing():
    print("Интерактивная предобработка промта.")
    print("Напишите промт на русском и нажмите Enter.")
    print("Для выхода введите: exit / quit / выход\n")

    while True:
        raw = input("Введите промт: ").strip()

        if raw.lower() in ["exit", "quit", "выход"]:
            print("Выход из интерактивного режима.")
            break
        row = pd.Series({
            "prompts": raw,
            "prepared_reference": None
        })

        processed = preprocess_prompt_row(row)

        print("\nРезультат предобработки")
        print(f"raw_prompt: {processed['raw_prompt']}")
        print(f"cleaned_prompt: {processed['cleaned_prompt']}")
        print(f"platform: {processed['platform']}")
        print(f"goal: {processed['goal']}")
        print(f"brand_style: {processed['brand_style'][:80]}")
        print(f"prepared_reference: {processed['prepared_reference']}")

In [10]:
#interactive_prompt_preprocessing()

In [13]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [12]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1

In [14]:
print(model.device)

cuda:0


In [15]:
SYSTEM_PROMPT = f"""
You are a Prompt Enhancer for an image generation system.

Your job:
- Take short, messy, mixed-language user prompts (mostly Russian).
- Normalize them into clean, detailed English prompts for image generation.
- Always follow the given brand visual style.

Default brand style:
{BRAND_STYLE_SHORT}

General rules:
1. Output ONLY the final enhanced prompt (no explanations, no comments).
2. Always respond in English.
3. Keep the original user intent (platform, goal, mood) but:
   - make the visual more premium, modern and minimalistic,
   - integrate the brand style (soft purple gradients, clean composition, premium digital aesthetic).
4. If platform/goal are given, adapt composition accordingly:
   - instagram_story / reels: vertical, strong focal point, works with overlaid text.
   - banner / website_visual: more space for text, clear hierarchy.
   - product_shot: focus on product, clean background.
   - promotion / sale: strong typography, high contrast, clear call-to-action space.
5. DO NOT mention 'brand style' or color codes explicitly in the prompt.
   Instead, implicitly apply them as visual characteristics.
"""

In [ ]:
def build_user_content(
    raw_prompt: str,
    cleaned_prompt: str | None = None,
    platform: str | None = None,
    goal: str | None = None,
    brand_style: str | None = None
) -> str:

    cleaned_prompt = cleaned_prompt or raw_prompt
    brand_style = brand_style or BRAND_STYLE_SHORT

    return (
        "You will receive a short user prompt in Russian (sometimes slangy) "
        "and some structured metadata. Rewrite it as a single, detailed, "
        "English image-generation prompt.\n\n"
        f"Raw prompt (ru): {raw_prompt}\n"
        f"Cleaned prompt (ru): {cleaned_prompt}\n"
        f"Detected platform: {platform}\n"
        f"Detected goal: {goal}\n"
        f"Brand style description: {brand_style}\n\n"
        "Return ONLY the enhanced English prompt, no explanations."
        )

def enhance_prompt(
    raw_prompt: str,
    cleaned_prompt: str | None = None,
    platform: str | None = None,
    goal: str | None = None,
    brand_style: str | None = None,
    max_new_tokens: int = 200
) -> str:

    user_content = build_user_content(
        raw_prompt=raw_prompt,
        cleaned_prompt=cleaned_prompt,
        platform=platform,
        goal=goal,
        brand_style=brand_style,
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)

    with torch.no_grad():
        out_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id
        )

    gen_ids = out_ids[0, input_ids.shape[-1]:]
    enhanced = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return enhanced

def enhance_row(row):
    return enhance_prompt(
        raw_prompt=row["raw_prompt"],
        cleaned_prompt=row["cleaned_prompt"],
        platform=row["platform"],
        goal=row["goal"],
        brand_style=row["brand_style"]
    )


In [17]:
def interactive_enhance():
    print("Введите сырой промт (или пустую строку для выхода):")
    while True:
        raw = input("> ").strip()
        if not raw:
            print("Выход.")
            break

        cleaned = normalize_case(clean_text(raw))
        platform = detect_platform(cleaned)
        goal = detect_goal(cleaned)

        enhanced = enhance_prompt(
            raw_prompt=raw,
            cleaned_prompt=cleaned,
            platform=platform,
            goal=goal,
            brand_style=BRAND_STYLE_SHORT
        )

        print("\nРезультат")
        print(f"raw_prompt: {raw}")
        print(f"cleaned_prompt: {cleaned}")
        print(f"platform: {platform}")
        print(f"goal: {goal}")
        print(f"enhanced_prompt: {enhanced}")

In [18]:
test_raw = "Баннер для магазина единорогов со скидкой 20%"
test_clean = normalize_case(clean_text(test_raw))
test_platform = detect_platform(test_clean)
test_goal = detect_goal(test_clean)

print("cleaned:", test_clean)
print("platform:", test_platform)
print("goal:", test_goal)

enhanced = enhance_prompt(
    raw_prompt=test_raw,
    cleaned_prompt=test_clean,
    platform=test_platform,
    goal=test_goal,
    brand_style=BRAND_STYLE_SHORT
)

print("ENHANCED PROMPT:", enhanced)

cleaned: Баннер для магазина единорогов со скидкой 20%
platform: banner
goal: promotion
ENHANCED PROMPT: Create a modern minimalistic premium banner promoting a store that sells unicorns, featuring a bright purple gradient at the top of the design, integrating a deep violet shadow effect, a clean and professional layout, incorporating digital glossy accents to enhance the feel of luxury and exclusivity, highlighting the offer of a 20% discount prominently, ensuring the logo of the store is visible and the message "Limited Time Offer" is bold and prominent, making sure the overall design supports a high-end commercial aesthetic with a touch of elegance.


In [ ]:
#interactive_enhance()

Введите сырой промт (или пустую строку для выхода):

Результат
raw_prompt: Баннер для магазина единорогов со скидкой 20%
cleaned_prompt: Баннер для магазина единорогов со скидкой 20%
platform: banner
goal: promotion
enhanced_prompt: A modern minimalist promotional banner featuring a cute unicorn in a vibrant shade of deep violet against a backdrop of soft purple gradients that evoke luxury and elegance. The banner is designed to stand out with its high-end commercial aesthetic, highlighting the store's unique products through a clear call-to-action, offering a discount of 20%.


Результат
raw_prompt: выход
cleaned_prompt: Выход
platform: generic
goal: general_visual
enhanced_prompt: A modern, minimalist, premium digital illustration featuring a person exiting from a dark room through a door with a soft purple gradient background that transitions to deeper violet shades at the edges, highlighting the digital glossiness of the scene, ensuring a clean and sleek composition that is visuall

In [20]:
df_results = df_processed.copy()
df_results["enhanced_prompt"] = None

for i, row in tqdm(df_results.iterrows(), total=len(df_results)):
    try:
        enhanced = enhance_prompt(
            raw_prompt=row["raw_prompt"],
            cleaned_prompt=row["cleaned_prompt"],
            platform=row["platform"],
            goal=row["goal"],
            brand_style=row["brand_style"]
        )
        df_results.at[i, "enhanced_prompt"] = enhanced
    except Exception as e:
        print(f"\nОшибка на строке {i}: {e}")
        df_results.at[i, "enhanced_prompt"] = None

100%|██████████| 100/100 [38:06<00:00, 22.86s/it]


In [21]:
df_results.head()

,raw_prompt,cleaned_prompt,platform,goal,brand_style,prepared_reference,cleaned_human,enhanced_prompt
0,красивая картинка для сторис,красивая картинка для сторис,instagram_story,general_visual,"modern minimalistic premium visual, soft purpl...","Create a clean, visually appealing Instagram S...",Красивая картинка для сторис,"A modern, minimalist, and premium Instagram St..."
1,сделать фон поярче,сделать фон поярче,generic,background,"modern minimalistic premium visual, soft purpl...","Design a bright, vibrant background with smoot...",Сделать фон поярче,"make the background clearer/smoothen, use soft..."
2,фотка для рекламы одежды,фотка для рекламы одежды,generic,general_visual,"modern minimalistic premium visual, soft purpl...",Produce a stylish fashion advertisement image ...,Фотка для рекламы одежды,"A high-end, modern, and minimalist advertiseme..."
3,стильный баннер срочно,стильный баннер срочно,banner,branding,"modern minimalistic premium visual, soft purpl...","Generate a sleek, modern promotional banner wi...",Стильный баннер срочно,"A stylish banner urgently, modern minimalistic..."
4,летняя распродажа визуал,летняя распродажа визуал,generic,general_visual,"modern minimalistic premium visual, soft purpl...","Create a summer sale visual with warm colors, ...",Летняя распродажа визуал,A vibrant summer clearance event featuring sty...


In [23]:
cols_to_show = [
    "raw_prompt", "enhanced_prompt"
]

df_results[cols_to_show].head(10)

,raw_prompt,enhanced_prompt
0,красивая картинка для сторис,"A modern, minimalist, and premium Instagram St..."
1,сделать фон поярче,"make the background clearer/smoothen, use soft..."
2,фотка для рекламы одежды,"A high-end, modern, and minimalist advertiseme..."
3,стильный баннер срочно,"A stylish banner urgently, modern minimalistic..."
4,летняя распродажа визуал,A vibrant summer clearance event featuring sty...
5,фото девушки в бренд стиле,photo of a girl in brand style; modern minimal...
6,фирменные цвета добавить,"Add brand colors to the design, incorporating ..."
7,промо под новый продукт,promotional campaign for a new product featuri...
8,картинка под акции,"A high-end, modern minimalist commercial image..."
9,атмосферная фотка магазина,"A serene atmosphere photo of a store, featurin..."


In [24]:
df_results["len_raw"] = df_results["raw_prompt"].str.split().str.len()
df_results["len_enh"] = df_results["enhanced_prompt"].fillna("").str.split().str.len()

print("Средняя длина исходного промта:", df_results["len_raw"].mean())
print("Средняя длина улучшенного промта:", df_results["len_enh"].mean())

Средняя длина исходного промта: 3.72
Средняя длина улучшенного промта: 59.02


In [26]:
keywords = ["minimalist", "modern", "premium", "gradient", "soft purple", "clean composition"]

for kw in keywords:
    share = df_results["enhanced_prompt"].fillna("").str.contains(kw, case=False).mean()
    print(f"{kw}: встречается в {share*100:.1f}% промтов")

minimalist: встречается в 88.0% промтов
modern: встречается в 88.0% промтов
premium: встречается в 76.0% промтов
gradient: встречается в 98.0% промтов
soft purple: встречается в 92.0% промтов
clean composition: встречается в 56.0% промтов


In [27]:
df_results['enhanced_prompt'].to_csv(r"../datasets/check_promt.csv")

# Демонстрация пайплайна дообучения с помощью LORA

In [29]:
train_data = []

for _, row in df_results.iterrows():
    if isinstance(row["enhanced_prompt"], str) and len(row["enhanced_prompt"]) > 10:
        example = {
            "instruction": "Enhance this user prompt according to the brand visual style.",
            "input": row["raw_prompt"],
            "output": row["enhanced_prompt"]
        }
        train_data.append(example)

with open("train_prompts.jsonl", "w", encoding="utf-8") as f:
    for ex in train_data:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

len(train_data)

100

In [32]:
dataset = load_dataset("json", data_files="train_prompts.jsonl")["train"]

Generating train split: 100 examples [00:00, 801.90 examples/s]


In [43]:
small_ds = dataset.select(range(32))

In [44]:
def format_example(example):
    text = (
        "<|system|>You enhance prompts to premium brand style.\n"
        f"<|user|>{example['input']}\n"
        f"<|assistant|>{example['output']}"
    )
    return {"text": text}

In [45]:
small_formatted = small_ds.map(format_example)
print("Пример format_example:", small_formatted[0]["text"][:300])

Map: 100%|██████████| 32/32 [00:00<00:00, 1031.29 examples/s]

Пример format_example: <|system|>You enhance prompts to premium brand style.
<|user|>красивая картинка для сторис
<|assistant|>A modern, minimalist, and premium Instagram Story featuring a stunning, soft purple gradient background with subtle deep violet shadows, clean lines, and a touch of digital gloss to create a high-


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [46]:
def tokenize(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized = small_formatted.map(
    tokenize,
    batched=True,
    remove_columns=small_formatted.column_names,
)

Map: 100%|██████████| 32/32 [00:00<00:00, 820.99 examples/s]


In [48]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model_lora = get_peft_model(model, lora_config)
model_lora.print_trainable_parameters()

trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


c:\proging\pyfiles\MFDP\MFDP\.venv\Lib\site-packages\peft\mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\proging\pyfiles\MFDP\MFDP\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [49]:
training_args = TrainingArguments(
    output_dir="./qwen-lora-enhanced",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    max_steps=50,
    learning_rate=2e-4,
    logging_steps=5,
    fp16=True,
    save_steps=0,
)

In [51]:
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized
)

trainer.train()

Step,Training Loss
5,0.624000
10,0.598400
15,0.407600
20,0.489400
25,0.362400
30,0.425600
35,0.373900
40,0.421000
45,0.398800
50,0.341700


TrainOutput(global_step=50, training_loss=0.4442736554145813, metrics={'train_runtime': 156.3528, 'train_samples_per_second': 0.64, 'train_steps_per_second': 0.32, 'total_flos': 403206045696000.0, 'train_loss': 0.4442736554145813, 'epoch': 3.125})

# Анализ ноутбука

## Модель

В качестве оновной модели используется **Qwen2.5-1.5B-Instruct**, загруженная в 4-битах, т.к. она компактная, помещается на GPU 6–8 GB, качественно работает на текстовых преобразованиях.

Enhance Prompt работает так, сырой промт + очищенный промт + платформа + цель + бренд-стиль передаются на модель и она генерирует профессиональный английский промт.

Почему используется готовая модель, а не свое решение? Ответ простой, не реально обучить NLP модель (LLM) которая будет хоть чуть приближена даже к качеству готовых бесплатных моделей, локально.

## Постобработка результатов

### Количественные метрики

|Метрика|Значение|
|-|-|
|Средняя длина исходного промта|3.72 слова|
|Средняя длина улучшенного промта|59.02 слов|

Рост длины больше чем в 10 раз означает, что значительно добавились:

- композиции
- цвета
- структуры
- контекста платформы
- эмоционального тона
- фирменного стиля

|Keyword|Keyword|
|-|-|
|minimalist|88%|
|modern|88%|
|premium|76%|
|gradient|98%|
|soft purple|92%|
|clean composition|56%|

Из чего понятно, что модель неплохо перенимает фирменный стиль и стабильно его применяет.

### Качественный анализ результатов

Сильные стороны модели

- Понимает платформу и адаптирует промт под формат
- Автоматически добавляет фирменный визуальный язык
- Генерирует очень богатые, подробные, развернутые промты
- Подходит для генераторов изображений

#### Пример улучшения

Было:
красивая картинка для сторис

Стало:
A modern, minimalist, premium Instagram Story featuring a soft purple gradient background… clean lines… digital gloss… strong focal point…

Исходя из примера выше множества других, можно понять, что это отлично готовый промт для генерации.

### Ошибки модели

1. Служебные фразы ("Cleaned up version", “Enhanced Prompt”)

Модель иногда игнорирует инструкцию “output only final prompt”.

2. Склейка токенов

Пример:

Premiumpremiumpremiumimagegenerationprompt...

3. Повторы фрагментов
View from above 
View from above
View from above

4. Иногда промты слишком короткие

## Не завершенный на данный момент LoRA-тюнинг

К сожалению во время изучения и работы с лора тюнингом у меня кончились часы на кагле, в данном ноутбуке представлен только пайплайн работы с лора тюнингом, но даже по нему можно предаоложить следующее.

Обучение (50 шагов) показало снижение лосса:

|Step|Training Loss|
|-|-|
|5|0.6240|
|10|0.5984|
|15|0.4076|
|20|0.4894|
|25|0.3624|
|30|0.4256|
|35|0.3739|
|40|0.4210|
|45|0.3988|
|50|0.3417|

Это может означать, что:

- LoRA может устранить недостатки (повторы, служебные фразы)
- Можно добиться идеально управляемого фирменного Prompt Enhancer
